In [1]:
import os
import pickle
import torch
import numpy as np
from numpy.random import RandomState
from math import ceil
from collections import OrderedDict

from scipy import signal
from scipy.signal import get_window
import librosa
from librosa.filters import mel
import soundfile as sf
from IPython.display import Audio

import matplotlib.pyplot as plt 

from model_bl import D_VECTOR
from model_vc import Generator
from synthesis import build_model
from synthesis import wavegen

In [2]:
# !wget https://github.com/nicolalandro/autovc/releases/download/0.1/3000000-BL.ckpt
# !wget https://github.com/nicolalandro/autovc/releases/download/0.1/autovc.ckpt
# !wget https://github.com/nicolalandro/autovc/releases/download/0.1/checkpoint_step001000000_ema.pth

# Read Audio

In [3]:
# path1="wavs/p225/p225_003.wav"
# path2="wavs/i300/galatea_01_barrili_f000001.wav"
# path2="wavs/i300/galatea_01_barrili_f000001.wav"
# path1="wavs/i301/imalavoglia_00_verga_f000002.wav"
# path1="/media/mint/Barracuda/Datasets/VoxCeleb/nas_data/wav/id10012/0AXjxNXiEzo/00001.wav"
# path2="/media/mint/Barracuda/Datasets/VoxCeleb/nas_data/wav/id10002/6WO410QOeuo/00001.wav"
path1="/media/mint/Barracuda/Datasets/Shah_Vocal_URDU/v1_Pak_70/id0001/English/A8Ch7gJalls/00000.wav"
path2="/media/mint/Barracuda/Datasets/Shah_Vocal_URDU/v1_Pak_70/id0002/Urdu/9mXNiGMx8rw/00000.wav"

In [4]:
Audio(path1)

In [5]:
Audio(path2)

# Compute spectrogram
Generate spectrogram by a Short Time Futrie Transform (STFT)

In [6]:
def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = signal.butter(order, normal_cutoff, btype='high', analog=False)
    return b, a
    
    
def pySTFT(x, fft_length=1024, hop_length=256):
    
    x = np.pad(x, int(fft_length//2), mode='reflect')
    
    noverlap = fft_length - hop_length
    shape = x.shape[:-1]+((x.shape[-1]-noverlap)//hop_length, fft_length)
    strides = x.strides[:-1]+(hop_length*x.strides[-1], x.strides[-1])
    result = np.lib.stride_tricks.as_strided(x, shape=shape,
                                             strides=strides)
    
    fft_window = get_window('hann', fft_length, fftbins=True)
    result = np.fft.rfft(fft_window * result, n=fft_length).T
    
    return np.abs(result)  

In [7]:
mel_basis = mel(16000, 1024, fmin=90, fmax=7600, n_mels=80).T
min_level = np.exp(-100 / 20 * np.log(10))
b, a = butter_highpass(30, 16000, order=5)

In [12]:
def prepare_spectrogram(path, rd_int=None):
    x, fs = sf.read(path)
    y = signal.filtfilt(b, a, x)
    if rd_int is None:
        rd_int = int(path.split('/')[-2][1:])
    prng = RandomState(rd_int) # cosa vuol dire?
    wav = y * 0.96 + (prng.rand(y.shape[0])-0.5)*1e-06
    D = pySTFT(wav).T
    D_mel = np.dot(D, mel_basis)
    D_db = 20 * np.log10(np.maximum(min_level, D_mel)) - 16
    S = np.clip((D_db + 100) / 100, 0, 1)  
    S = S.astype(np.float32)
    return S

In [13]:
s1 = prepare_spectrogram(path1, rd_int=0)
print(s1.shape)
s2 = prepare_spectrogram(path2, rd_int=1)
print(s2.shape)

(828, 80)
(373, 80)


In [14]:
# we can plot these spectrograms?

# Speaker Encoder
Use model_bl to encode the spectrograms

In [15]:
C = D_VECTOR(dim_input=80, dim_cell=768, dim_emb=256).eval().cuda()
c_checkpoint = torch.load('3000000-BL.ckpt')
new_state_dict = OrderedDict()
for key, val in c_checkpoint['model_b'].items():
    new_key = key[7:]
    new_state_dict[new_key] = val
C.load_state_dict(new_state_dict)
num_uttrs = 10
len_crop = 128

In [16]:
def process_speacker(tmp):
    left = np.random.randint(0, tmp.shape[0]-len_crop)
    melsp = torch.from_numpy(tmp[np.newaxis, left:left+len_crop, :]).cuda()
    emb = C(melsp)
    return emb.detach().squeeze().cpu().numpy()

In [17]:
emb1 = process_speacker(s1)
print(emb1.shape)
emb2 = process_speacker(s2)
print(emb2.shape)

(256,)
(256,)


# AutoVC
It use the model_vc to decode both data: input and style

In [18]:
def pad_seq(x, base=32):
    len_out = int(base * ceil(float(x.shape[0])/base))
    len_pad = len_out - x.shape[0]
    assert len_pad >= 0
    return np.pad(x, ((0,len_pad),(0,0)), 'constant'), len_pad

In [19]:
device = 'cuda:0'
# G = Generator(32,256,512,32).eval().to(device)
# g_checkpoint = torch.load('autovc.ckpt', map_location=device)
# G.load_state_dict(g_checkpoint['model'])

G = Generator(32,256,512,32).eval().to(device)
# g_checkpoint = torch.load("/media/mint/Barracuda/Models/autovc/Models/v1_Pak_70/generator_ita_only_200000_circa.pth", map_location=device)
g_checkpoint = torch.load("/media/mint/Barracuda/Models/autovc/Models/v1_Pak_70/generator.pth", map_location=device)
G.load_state_dict(g_checkpoint)

<All keys matched successfully>

In [20]:
def prepare_input(s1, emb1, emb2):
    x_org, len_pad = pad_seq(s1)
    uttr_org = torch.from_numpy(x_org[np.newaxis, :, :]).to(device)
    emb_org = torch.from_numpy(emb1[np.newaxis, :]).to(device)
    
    emb_trg = torch.from_numpy(emb2[np.newaxis, :]).to(device)
    
    with torch.no_grad():
            _, x_identic_psnt, _ = G(uttr_org, emb_org, emb_trg)
    if len_pad == 0:
        uttr_trg = x_identic_psnt[0, 0, :, :].cpu().numpy()
    else:
        uttr_trg = x_identic_psnt[0, 0, :-len_pad, :].cpu().numpy()
    return uttr_trg

In [21]:
spect_vc1 = prepare_input(s1, emb1, emb2)
print(spect_vc1.shape)

(828, 80)


# Vocoder
Generate .wav audio file from generated spectrogram

In [22]:
device = torch.device("cuda")
model = build_model().to(device)
checkpoint = torch.load("checkpoint_step001000000_ema.pth")
model.load_state_dict(checkpoint["state_dict"])

<All keys matched successfully>

In [ ]:
file_output = "a_b.wav"
c = spect_vc1
waveform = wavegen(model, c=c)
sf.write(file_output, waveform, 16000, 'PCM_24')

 70%|███████   | 149077/211968 [15:54<07:00, 149.42it/s]

In [ ]:
Audio(file_output)